Скачаем датасет с короткими постами sentiment140

In [1]:
import requests

url = "https://code.s3.yandex.net/deep-learning/tweets.txt"
response = requests.get(url)

with open("data/raw_dataset.csv", "wb") as f:
    f.write(response.content)


В data_utils.py
- удалим ссылкина сайты
- приведем к нижнем регистру
- удалим все, кроме латинских букв, цифр и пробелов
- удалим дублирующиеся пробелы, пробелы по краям
- токенизируем текст
- разобьем на 3 датасета: тренировочный, валидационный, тестовый

"очистим" текст

In [ ]:
import importlib
import src.data_utils

importlib.reload(src.data_utils)

from src.data_utils import preprocess_dataset

preprocess_dataset("data/raw_dataset.csv", "data/dataset_processed.csv")


токенизируем текст

In [5]:
import importlib
import src.data_utils

importlib.reload(src.data_utils)


from src.data_utils import tokenize_dataset

tokenize_dataset(
    "data/dataset_processed.csv",
    "data/tokenized_dataset.csv"
)


Разобьём на 3 датасета: тренировочный (80%), валидационный (10%), тестовый (10%)

In [6]:
import importlib
import src.data_utils

importlib.reload(src.data_utils)

from src.data_utils import split_dataset

split_dataset(
    input_path="data/tokenized_dataset.csv",
    train_path="data/train.csv",
    val_path="data/val.csv",
    test_path="data/test.csv"
)


In [ ]:
В next_token_dataset.py
Создадим даталоадеры и train.csv, val.csv, test.csv

In [7]:
from src.next_token_dataset import make_dataloader
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
pad_id = tokenizer.pad_token_id  # у bert-base-uncased это 0

batch_size = 64
max_len = 128

_, train_loader = make_dataloader("data/train.csv", batch_size=batch_size, shuffle=True,  max_len=max_len, pad_id=pad_id)
_, val_loader   = make_dataloader("data/val.csv",   batch_size=batch_size, shuffle=False, max_len=max_len, pad_id=pad_id)
_, test_loader  = make_dataloader("data/test.csv",  batch_size=batch_size, shuffle=False, max_len=max_len, pad_id=pad_id)

print("train batches:", len(train_loader))
print("val batches:", len(val_loader))
print("test batches:", len(test_loader))


train batches: 20015
val batches: 2502
test batches: 2502


In [8]:
batch = next(iter(train_loader))

print("input_ids:", batch["input_ids"].shape)
print("targets:", batch["targets"].shape)
print("lengths:", batch["lengths"].shape)

# посмотрим на один пример в батче (без паддинга)
i = 0
L = batch["lengths"][i].item()

x_ids = batch["input_ids"][i, :L].tolist()
y_ids = batch["targets"][i, :L].tolist()

print("X tokens:", tokenizer.convert_ids_to_tokens(x_ids[:20]))
print("Y tokens:", tokenizer.convert_ids_to_tokens(y_ids[:20]))

# sanity-check: Y должен быть X, сдвинутым на 1
print("Shift OK:", x_ids[1:] == y_ids[:-1])


input_ids: torch.Size([64, 31])
targets: torch.Size([64, 31])
lengths: torch.Size([64])
X tokens: ['[CLS]', 'my', '##sw', '##eet', '##mel', '##od', '##y', 'nice', 'tie', 'i', 'love', 'the', 'green']
Y tokens: ['my', '##sw', '##eet', '##mel', '##od', '##y', 'nice', 'tie', 'i', 'love', 'the', 'green', '[SEP]']
Shift OK: True
